## Collaborative Filtering

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from recsys_utils import *
from public_tests import *

### Dataset

In [ ]:
x_train, w_train, b_train, num_movies, num_features, num_users=load_precalc_params_small()
y_train, r_train=load_ratings_small()

print(f"X: {x_train.shape}")
print(f"Y: {y_train.shape}")
print(f"R: {r_train.shape}")
print(f"W: {w_train.shape}")
print(f"b: {b_train.shape}")
print(f"Movies: {num_movies}")
print(f"Features: {num_features}")
print(f"Users: {num_users}")

In [ ]:
# Finding average rating for movie 1

tsmean=np.mean(y_train[0, r_train[0, :].astype(bool)])
print(f"Average Rating of Movie 1: {tsmean}/5")

### Cost Function

In [ ]:
def cost_function(X, W, b, Y, R, lambda_):
  nm, nu=Y.shape
  J=0

  for j in range(nu):
    w=W[j, :]
    b_j=b[0, j]
    for i in range(nm):
      x=X[i, :]
      y=Y[i, j]
      r=R[i, j]
      J+=r*np.square((np.dot(w, x)+b_j)-y)

  J+=(lambda_)*(np.sum(np.square(W))+np.sum(np.square(X)))
  J=J/2

  return J

In [ ]:
test_cofi_cost_func(cost_function)

In [ ]:
num_users_test=4
num_movies_test=5
num_features_test=3

x_test=x_train[:num_movies_test, :num_features_test]
w_test=w_train[:num_users_test, :num_features_test]
b_test=b_train[0, :num_users_test].reshape(1, -1)
y_test=y_train[:num_movies_test, :num_users_test]
r_test=r_train[:num_movies_test, :num_users_test]

J=cost_function(x_test, w_test, b_test, y_test, r_test, 0)
print(f"Cost function value: {J:0.4f}")

In [ ]:
# Cost function for whole dataset
J=cost_function(x_train, w_train, b_train, y_train, r_train, 1.5)
print(f"Cost function with regularization: {J:0.2f}")

### Vectorized Implementation of Cost Function

In [ ]:
def cost_function_vect(X, W, b, Y, R, lambda_):
  j=((tf.linalg.matmul(X, tf.transpose(W))+b)-Y)*R
  J=(0.5*tf.reduce_sum(j**2))+((lambda_/2)*(tf.reduce_sum(W**2)+tf.reduce_sum(X**2)))
  return J

In [ ]:
J=cost_function_vect(x_test, w_test, b_test, y_test, r_test, 1.5)
print(f"Cost function value: {J:0.4f}")

### Learning Movie Recommendations

In [ ]:
movieList, movieList_df=load_Movie_List_pd()

In [ ]:
my_ratings=np.zeros(num_movies)

In [ ]:
movieList_df

In [ ]:
# Randomly assigning ratings to some movies

my_ratings[2700]=5
my_ratings[2609]=2
my_ratings[929]=5
my_ratings[246]=5
my_ratings[2716]=3
my_ratings[1150]=5
my_ratings[382]=2
my_ratings[366]=5
my_ratings[622]=5
my_ratings[988]=3
my_ratings[2925]=1
my_ratings[2937]=1
my_ratings[793]=5

In [ ]:
my_rated=[i for i in range(len(my_ratings)) if my_ratings[i]>0]

In [ ]:
print("New User Ratings")
for i in range(len(my_ratings)):
  if my_ratings[i]>0:
    print(f"Rated {my_ratings[i]} for {movieList_df.loc[i, "title"]}")

In [ ]:
y_train, r_train=load_ratings_small()
y_train=np.c_[my_ratings, y_train]
r_train=np.c_[(my_ratings!=0).astype(int), r_train]

In [ ]:
y_norm, y_mean=normalizeRatings(y_train, r_train)

In [ ]:
num_movies, num_users=y_train.shape
num_features=100

In [ ]:
tf.random.set_seed(1234)

W=tf.Variable(tf.random.normal((num_users, num_features), dtype=tf.float64), name='W')
X=tf.Variable(tf.random.normal((num_movies, num_features), dtype=tf.float64), name='X')
b=tf.Variable(tf.random.normal((1, num_users), dtype=tf.float64), name='b')

optimizer=keras.optimizers.Adam(learning_rate=1e-1)

In [ ]:
iterations=200
lambda_=1

for iter in range(iterations):
  # To record the operations used to compute the cost
  with tf.GradientTape() as tape:
    cost_value=cost_function_vect(X, W, b, y_norm, r_train, lambda_)

  # The gradients of the trainable variables w.r.t to the loss
  gradients=tape.gradient(cost_value, [X, W, b])

  # Apply gradient descent
  optimizer.apply_gradients(zip(gradients, [X, W, b]))

  if iter%20==0:
    print(f"Training loss at iteration {iter}: {cost_value:0.2f}")

### Making Recommendations

In [ ]:
# Making predictions using trained weights and biases
p=np.matmul(X.numpy(), np.transpose(W.numpy()))+b.numpy()

# Restore the mean
pm=p+y_mean

my_predictions=pm[:, 0]

ix=tf.argsort(my_predictions, direction='DESCENDING')

for i in range(17):
  j=ix[i]
  if j not in my_rated:
    print(f"Predicted Rating: {my_predictions[j]:0.2f} for movie {movieList[j]}")

print("\n\n")
for i in range(len(my_ratings)):
  if my_ratings[i]>0:
    print(f"Original Rating: {my_ratings[i]} Predicted Rating: {my_predictions[i]:0.2f} for movie {movieList[i]}")

In [ ]:
filter=(movieList_df['number of ratings']>20)
movieList_df["predicted rating"]=my_predictions
movieList_df=movieList_df.reindex(columns=['predicted rating', 'mean rating', 'number of ratings', 'title'])
movieList_df.loc[ix[:300]].loc[filter].sort_values('mean rating', ascending=False)

## 🚫 Limitations: The Cold Start Problem

### What is the **Cold Start Problem**?

The cold start problem occurs when the system **doesn’t have enough data** about **new users** or **new items**, which makes it hard for collaborative filtering to generate accurate recommendations.

### Two Scenarios of the Cold Start Problem:

#### 1. **New Item Problem**

* **Challenge**: How to **rank new items** (like a new movie, product, or book) that **few or no users have rated**?
* **Why it's a problem**: Collaborative filtering works by looking at user interactions (ratings, clicks). If no one has rated or interacted with the new item, the system has no way to recommend it intelligently.

#### 2. **New User Problem**

* **Challenge**: How to **show relevant recommendations to new users** who have rated/interacted with **very few items**?
* **Why it's a problem**: Without enough information about a user’s preferences, it’s hard to identify similar users or recommend items accurately.

These two are the core aspects of the cold start problem.

---

## 🧠 Possible Solution: Use Side Information

Since collaborative filtering struggles with cold starts, a common solution is to **supplement it with additional (side) information** about items or users.

### 📦 Side Information About Items:

You can enrich item data with content-based features like:

* **Genre** (e.g., for movies: action, drama)
* **Movie stars or authors**
* **Studio or publisher**
* **Release year, language, etc.**

This can help recommend items even if they haven't been rated much, by matching content to the user’s known preferences.

### 👤 Side Information About Users:

You can also gather metadata about users such as:

* **Demographics** (age, gender, location)
* **Explicitly expressed preferences**
* **Behavioral patterns (e.g., time of use, browsing habits)**

With this data, you can make early predictions for new users based on demographic or behavioral similarities to existing users.

---

## ✅ Summary

| Limitation            | Why It Happens                           | How to Mitigate                        |
| --------------------- | ---------------------------------------- | -------------------------------------- |
| Cold Start – New Item | No user has interacted with the item     | Use item metadata (genre, stars, etc.) |
| Cold Start – New User | User hasn’t interacted with enough items | Use user demographics/preferences      |